In [1]:
from light_curves import LightCurve
from exposures import Calexp
from task import Run

In [2]:
process = Run()
params = [{"t_0": 60700.5, "t_E": 30, "u_0": 0.5, "m_base": 19.5},
          {"t_0": 61000.5, "t_E": 4, "u_0": 0.5, "m_base": 22.5}]
# for p in params:
#     process.add_lc(p, dist=0.2)
process.add_lc(params[0], ra=57.67048780800139, dec=-32.49757834967935)
process.add_lc(params[1], dist=0.2)
process.log_task("Add and simulate light curves")
process.inj_lc

(ra,dec) = (57.67048780800139, -32.49757834967935)/nHTM_ID = 9871863798730 - HTM_level=20 (bounded by a circle of radius ~0.27 arcsec.)
                    Found 150 calexps


/home/karennowo/DP0_microlensing/light_curves.py:113: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.data = pd.concat([self.data, new_data], ignore_index=True)


[LightCurve (57.67048780800139, -32.49757834967935) - Band i - Points: 150,
 LightCurve (57.806160468725096, -32.63614307434996) - Band i - Points: 150]

In [3]:
process.inject_task()
schema = process.measure_task()
for j, dataId in enumerate(process.calexp_dataIds[:10]):
    calexp = Calexp(dataId)
    injected_exp, injected_points = process.inj_calexp(calexp)
    if injected_exp != None:
        sources = process.measure_calexp(injected_exp, schema)
        flux, flux_err = process.find_flux(sources, add_to_calexp=dataId)
        mag, mag_err = add_mag(self, value, value_err, dataId, exposure = injected_exp)
        print(f"ra = {ra_deg}, dec = {dec_deg}")
        print("Measured ", mag, mag_err)
        print("Injected ", lc.data["mag"][j])


Points injected: 2


[87301.42792527] [array([87301.42792527])] [558.70891799] [array([558.70891799])]
0   NaN
Name: flux, dtype: float64
[8001.57496116 6720.87808071] [array([87301.42792527]), array([8001.57496116, 6720.87808071])] [378.14056184 377.39274254] [array([558.70891799]), array([378.14056184, 377.39274254])]
Series([], Name: flux, dtype: object)


ValueError: Must have equal len keys and value when setting with an iterable

In [ ]:
sources.asAstropy().columns
flux = sources[abs(sources["coord_ra"]-57.67048780800139*np.pi/180)< 0.0000001]["base_PsfFlux_instFlux"]
flux_err = sources[abs(sources["coord_ra"]-57.67048780800139*np.pi/180)< 0.0000001]["base_PsfFlux_instFluxErr"]

In [9]:
lc = process.inj_lc[1]
import numpy as np
flux = sources[abs(sources["coord_ra"]-lc.ra*np.pi/180)< 0.0000001]["base_PsfFlux_instFlux"]

flux_err = sources[abs(sources["coord_ra"]-lc.ra*np.pi/180)< 0.0000001]["base_PsfFlux_instFluxErr"]
flux, flux_err

(array([5353.62071767, 1748.9515086 ]), array([], dtype=float64))

In [12]:
sources[abs(sources["coord_ra"]-lc.dec*np.pi/180)< 0.000001]["base_PsfFlux_instFluxErr"]

array([], dtype=float64)

In [31]:
photoCalib = calexp.getPhotoCalib()
measure = photoCalib.instFluxToMagnitude(flux, flux_err)
measure

TypeError: instFluxToMagnitude(): incompatible function arguments. The following argument types are supported:
    1. (self: lsst.afw.image.PhotoCalib, instFlux: float, point: lsst.geom.Point2D) -> float
    2. (self: lsst.afw.image.PhotoCalib, instFlux: float) -> float
    3. (self: lsst.afw.image.PhotoCalib, instFlux: float, instFluxErr: float, point: lsst.geom.Point2D) -> lsst.afw.image.Measurement
    4. (self: lsst.afw.image.PhotoCalib, instFlux: float, instFluxErr: float) -> lsst.afw.image.Measurement
    5. (self: lsst.afw.image.PhotoCalib, sourceRecord: lsst.afw.table.SourceRecord, instFluxField: str) -> lsst.afw.image.Measurement
    6. (self: lsst.afw.image.PhotoCalib, sourceCatalog: lsst.afw.table.SourceCatalog, instFluxField: str) -> numpy.ndarray
    7. (self: lsst.afw.image.PhotoCalib, sourceCatalog: lsst.afw.table.SourceCatalog, instFluxField: str, outField: str) -> None

Invoked with: PhotoCalib(spatially constant with mean: 0.663395 error: 0.000191301), array([  2461.7